# MusicLab Notebook for Data Jockeying 

## 1. Importing libraries

In [ ]:
import os, fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rc('xtick', labelsize=14)     
mpl.rc('ytick', labelsize=14)
mpl.rcParams['font.size'] = 15
mpl.rcParams['legend.fontsize'] = 20
mpl.rcParams['figure.titlesize'] = 16

## 2. Directory management

In [ ]:
path = '/Users/cagrierdem/Desktop/dev/MusicLab/gig/20200710/'
datasets = ['data ('+str(nr)+')' for nr in range(0,13)]

In [ ]:
def ffind(pattern, path):
    result=[]
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [ ]:
for dat in ffind('*.csv', os.path.join(path, datasets[0])):
    if 'Motion' in dat:
        print('IMU √')
        motion_dat = dat
    elif 'geo' in dat:
        print('Geo √')
        geo_dat = dat

## 3. Reading & writing data into Pandas Dataframe

In [ ]:
df_motion = pd.read_csv(motion_dat, index_col=1)
df_motion.head(3)

![](app-imu.png)

In [ ]:
df_motion.iloc[:, 1:4].plot(figsize=(24,3))

In [ ]:
# First let's define the cut point. Let's say 100000

In [ ]:
#Then find the right index point
cut_point = (np.abs(df_motion.index-100000)).argmin()
cut_point

In [ ]:
#And cut the beginning & store it in a new variable for more flexibility
df_motion_edt  = df_motion.iloc[cut_point:, :]

##### We need to convert epoch to human-readable datetime:

In [ ]:
df_motion.timestamp = pd.to_datetime(df_motion.timestamp, unit='ms')
df_motion.head(3)

In [ ]:
duration = (df_motion.index[-1] - df_motion.index[0]) / 1000 / 60
duration

In [ ]:
freq = int(round(1 / np.mean(np.diff(df_motion.index)), 3) * 1000)
print('sr:', freq, 'Hz')

In [ ]:
time = np.linspace(0, len(df_motion)/freq, len(df_motion))

In [ ]:
#Raw acc data

In [ ]:
df_motion.iloc[:, 1:4].plot(figsize=(24,5))

In [ ]:
#Raw gyro data

In [ ]:
df_motion.iloc[:, 4:].plot(figsize=(24,5))

## 4. Pre-processing the data
### 4.1. Filtering
##### e.g. 4th order, zero-phase IIR lowpass or bandpass filter

In [ ]:
from scipy.signal import butter, lfilter

def butter_filt(data, lowcut, highcut, fs, order=4, btype='band'):
    nyq = fs / 2
    b, a = butter(order, [lowcut/nyq, highcut/nyq], btype=btype)
    y = lfilter(b, a, data)
    return y

In [ ]:
columns = list(df_motion.iloc[:, 1:4])
filtered=[]
for col in columns:
    filt = butter_filt(df_motion[col], 1, 10, fs=freq, order=4)
    filtered.append(filt)

### 4.2. Normalization

In [ ]:
def normalize(y, min_val=0):
    max_value = max(y)
    min_value = min(y)
    k = []
    for i in range(0, len(y)):
        if min_val == 0:
            k.append((y[i] - min_value) / (max_value - min_value))
        elif min_val == -1:
            k.append( 2*(y[i] - min_value) / (max_value - min_value)-1 )
    return np.array(k)

In [ ]:
filtnorm=[]
for ax in filtered:
    norm = normalize(ax, min_val=-1)
    filtnorm.append(norm)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(16,7))
labels = ['X', 'Y', 'Z']
colors = ['r', 'g', 'b']

for anr in range(3):
    ax[anr].plot(time, filtnorm[anr], label=labels[anr], linewidth=0.5, color=colors[anr], alpha=0.5)
    ax[anr].legend(loc='lower right', fontsize=14)
ax[0].set_title('Lateral - side to side')
ax[1].set_title('Vertical - up down')
ax[2].set_title('Frontal - forwards backwards')

fig.subplots_adjust(hspace=0.5)

## 5. Feature Extraction
### 5.1. Quantity of Motion (QoM)

In [ ]:
def qom(time, x, y, z):
    qom=[]
    for i in range(len(time)-1):
        id1 = sum((x[i],y[i],z[i]))
        id2 = sum((x[i+1],y[i+1],z[i+1]))
        diff = abs(id2-id1)
        qom.append(diff)
    return qom

#Note that this function can be written more elegantly

In [ ]:
#Filter QoM for a better representation
from scipy.signal import savgol_filter as savgol

#normalized QoM:
qom_n = normalize(qom(time,filtnorm[0],filtnorm[1],filtnorm[2])) 

# normalized (filtered) QoM trend:
qom_filt_order = 1
qom_win = 1999 #reduce for more resolution
qom_fn = normalize(savgol(qom(time,filtnorm[0],filtnorm[1],filtnorm[2]), qom_win, qom_filt_order)) 
                                                                

def plot_qoms(qom_n, qom_fn, fontsize=15):

    plt.figure(figsize=(24,7))
    plt.plot(time[:-1], qom_n, alpha=0.2, color='r', label='QoM')
    plt.plot(time[:-1], qom_fn, alpha=0.7, color='b', label='QoM Trend')
    plt.xticks(fontsize=15)
    plt.xlabel('Time (s)',fontsize=fontsize)
    plt.yticks(fontsize=15)
    plt.ylabel('Amplitude',fontsize=fontsize)
    plt.legend(loc='upper right', fontsize=fontsize)

In [ ]:
plot_qoms(qom_n, qom_fn)

In [ ]:
# Planar acceleration

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,7))

ax[0].plot(filtnorm[1], filtnorm[2], color='red', alpha=0.3, label='YZ')
ax[1].plot(filtnorm[0], filtnorm[2], color='blue', alpha=0.3, label='XZ')
ax[2].plot(filtnorm[0], filtnorm[1], color='green', alpha=0.3, label='XY')

[ax[n].legend(loc='upper right') for n in range(3)]

fig.subplots_adjust(wspace=.5)

### 5.2. Motion peaks

In [ ]:
import sensormotion as sm

def avg(axes_list):
    return np.sum([axes_list[0], axes_list[1], axes_list[2]], axis=0) / 3

peak_times, peak_values = sm.peak.find_peaks(time=time, signal=avg(filtnorm), peak_type='valley',
                                             min_val=0.6, min_dist=freq//2, 
                                             plot=True, fig_size=(24,5))

### 5.3. Periodograms

In [ ]:
#Note: Also try zero-crossings and autocorrelation

In [ ]:
from scipy import signal

periodograms=[]
for i in range(3):
    f, Pxx = signal.periodogram(filtnorm[i], fs=freq, window='hanning', scaling='spectrum')
    periodograms.append((f,Pxx))

In [ ]:
def plot_periods(periodogramz):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,7))
    colors = ['r','g', 'b']
    for idx,(f,Pxx) in enumerate(periodogramz):
        
        ax[idx].plot(f, Pxx, color=colors[idx], alpha=0.4)
        ax[idx].set_yscale('log') #Comment out for linear scale
        ax[idx].set_xlabel('Frequency (Hz)')
        ax[0].set_ylabel('Spectrum Amplitude')
          
    fig.subplots_adjust(wspace=.5)

In [ ]:
plot_periods(periodograms)

In [ ]:
def top_periods(f, Pxx):
    '''f=[sample frequencies]; Pxx=[power spectrum of x]'''
    tops={}
    top_fq_indices = np.flip(np.argsort(Pxx), 0)[0:3]
    freqs = f[top_fq_indices]
    power = Pxx[top_fq_indices]
    periods = 1 / np.array(freqs)
    
    for i in range(1,4):
        tops['Period_{} (secs)'.format(i)] = round(periods[i-1], 2)
        tops['Freq_{} (Hz)'.format(i)] = round(freqs[i-1], 3)
        tops['Tempo_{} (BPM)'.format(i)] = round(freqs[i-1]*60, 2)
        tops['Power_{} (A)'.format(i)] = format(power[i-1], 'f')
        
    return tops

In [ ]:
x_periods = top_periods(periodograms[0][0], periodograms[0][1])
y_periods = top_periods(periodograms[1][0], periodograms[1][1])
z_periods = top_periods(periodograms[2][0], periodograms[2][1])

In [ ]:
z_periods

## 6. Audio

In [ ]:
import librosa
from librosa import display
import IPython.display as ipd

sound, sr = librosa.load(os.path.join(path, 'data (0)/renick.wav'), duration=duration*60, sr=44100)
ipd.Audio(sound, rate=sr)

### 6.1. Source separation
### 6.2. Onset detection
### 6.3. Tempo tracking

In [ ]:
# Notes: Here I detect onset based on the decomposed percussive components. BUT, also use non-decomposed signal. 
# The outcome differs depending on the musical content
# In addition, dynamic tempo tracking is a better option for rhythmically non-conventional musics

In [ ]:
def onset_tempo(sound, sr=44100, hop_length=512, n_fft=2048, Normalize=True):
    perc = librosa.effects.percussive(sound) #6.1.
    onset_strength = librosa.onset.onset_strength(y=perc, sr=sr, hop_length=hop_length) #6.2.
    onset_strength_n = normalize(onset_strength)
    time_s = librosa.times_like(onset_strength, sr=sr, hop_length=hop_length, n_fft=n_fft)
    tempo = librosa.beat.tempo(onset_envelope=onset_strength, sr=sr) #6.3.
    if Normalize:
        return time_s, onset_strength_n, tempo
    else:
        return time_s, onset_strength, tempo

In [ ]:
audio_features = onset_tempo(sound)

In [ ]:
print('Tempo: %.i BPM'%(audio_features[2][0]))

In [ ]:
#Plot all together

In [ ]:
peak_values_n = normalize(peak_values)

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(16,7))

ax[0].plot(audio_features[0], audio_features[1], alpha=0.4, linewidth=2.5, label='Audio onset strength', color='blue')

ax[1].plot(peak_times, peak_values_n, alpha=0.4, linewidth=2.5, label='ACC peaks', color='red')

ax[2].plot(time[:-1], qom_fn, alpha=0.4, linewidth=2.5, label='QoM trend', color='green')

[ax[n].grid(True) for n in range(3)]
[ax[n].legend(loc='lower center') for n in range(3)]

## 7. Geo-location

In [ ]:
df_geo = pd.read_csv(geo_dat,index_col=0)
df_geo.head()

In [ ]:
from mpl_toolkits.basemap import Basemap
from geopy.geocoders import Nominatim

def get_address(lat, long, api='geoapiExercises'):
    geolocator = Nominatim(user_agent=api)
    location = geolocator.reverse(str(lat)+ ', ' + str(long))
    return location.address.split(', ')

def get_coordinates(place, api='geoapiExercises', timeout=5):
    geolocator = Nominatim(user_agent=api)
    location = geolocator.geocode(place, timeout=timeout)
    if not location:
        return None, None
    return location.latitude, location.longitude

def plot_map(latitude, longitude, figsize=(10,10), markersize=12, fontsize=20, verbose=False):
    address = get_address(latitude, longitude)
    if verbose:
        print(address)
    address = address[-3:]
    city_lat = get_coordinates(address[0])[0]
    city_lon = get_coordinates(address[0])[1]
    
    fig = plt.figure(figsize=figsize)
    m = Basemap(projection='lcc', resolution=None, 
                width=8E6, height=8E6,
                lat_0 = city_lat, lon_0 = city_lon)
    m.etopo(scale=1, alpha=0.5)
    x, y = m(longitude, latitude)
    plt.plot(x, y, 'ok', markersize=markersize)
    plt.text(x, y, address[0], fontsize=fontsize)

In [ ]:
plot_map(df_geo.latitude.values[0], df_geo.longitude.values[0], verbose=True)